In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from libpysal.weights import *

In [2]:
def xrast2W(rasterf, rook=True, **kwargs):
    """
    Create a W object from rasters(xarray.DataArray)
    
    Parameters
    ----------
    raster     : xarray.DataArray
                 raster file accessed using xarray.open_rasterio method
    rook       : boolean
                 type of contiguity. Default is rook. For queen, rook =False
    **kwargs   : keyword arguments
                 optional arguments for :class:`pysal.weights.W`
    Returns
    -------
    w    : libpysal.weights.W
           instance of spatial weights class W
    data : pandas.Series
           Values from `raster` as a vector of dimension (`w.n` x
           None) aligned with `w`            
    """
    w = lat2W(*rasterf[0].shape, rook=rook, **kwargs)
    if isinstance(rasterf, xr.DataArray):
        rasterf_masked = rasterf.where(rasterf.data != rasterf.nodatavals[0])
        rasterf_masked = rasterf_masked[0].data.flatten()
        id_order = np.argwhere(~np.isnan(rasterf_masked)).flatten()
        w = w_subset(w, id_order)
        data = rasterf_masked[~np.isnan(rasterf_masked)]
        data = pd.Series(data,index=id_order)
    return w, data

In [3]:
def xrast2SW(rasterf, criterion="rook"):
    """
    Generate a sparse W matrix from rasters(xarray.DataArray)
    
    Parameters
    ----------
    raster     : xarray.DataArray
                 raster file accessed using xarray.open_rasterio method
    rook       : {"rook", "queen"}
                 type of contiguity. Default is rook.
    Returns
    -------
    w    : scipy.sparse.csr_matrix
           instance of a scipy sparse matrix
    data : pandas.Series
           Values from `raster` as a vector of dimension (`w.n` x
           None) aligned with `w`            
    """

    sw = lat2SW(*rasterf[0].shape, criterion=criterion)
    if isinstance(rasterf, xr.DataArray):
        rasterf_masked = rasterf.where(rasterf.data != rasterf.nodatavals[0])
        rasterf_masked = rasterf_masked[0].data.flatten()
        sw = sw.multiply((~np.isnan(rasterf_masked))[np.newaxis].T)
        sw = sw.multiply((~np.isnan(rasterf_masked))).tocsr()
        id_order = np.argwhere(~np.isnan(rasterf_masked)).flatten()
        data = rasterf_masked[~np.isnan(rasterf_masked)]
        data = pd.Series(data,index=id_order)
    return sw, data

In [4]:
da = xr.open_rasterio('/home/magito/GSoC/test-notes/lux_ppp_2019.tif')
da = da[:,:30,:]
da

<xarray.DataArray (band: 1, y: 30, x: 940)>
array([[[-99999., -99999., ..., -99999., -99999.],
        [-99999., -99999., ..., -99999., -99999.],
        ...,
        [-99999., -99999., ..., -99999., -99999.],
        [-99999., -99999., ..., -99999., -99999.]]], dtype=float32)
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 50.18 50.18 50.18 50.18 ... 50.16 50.16 50.16 50.16
  * x        (x) float64 5.745 5.746 5.747 5.747 ... 6.525 6.526 6.527 6.527
Attributes:
    transform:      (0.0008333333297872345, 0.0, 5.744583325, 0.0, -0.0008333...
    crs:            +init=epsg:4326
    res:            (0.0008333333297872345, 0.0008333333295454553)
    is_tiled:       0
    nodatavals:     (-99999.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area

In [5]:
w, data = xrast2W(da)

In [6]:
from itertools import islice

def take(n, iterable):
    return list(islice(iterable, n))

take(10, w.neighbors.items())

[(332, [1272, 333]),
 (333, [1273, 332, 334]),
 (334, [1274, 333, 335]),
 (335, [336, 1275, 334]),
 (336, [1276, 335]),
 (1268, [2208, 1269]),
 (1269, [2209, 1268, 1270]),
 (1270, [2210, 1269, 1271]),
 (1271, [1272, 2211, 1270]),
 (1272, [1273, 1271, 2212, 332])]

In [7]:
data.head()

332    2.473892
333    2.568437
334    2.164773
335    1.874968
336    1.179762
dtype: float32

In [8]:
ws, data = xrast2SW(da)

In [9]:
ws[0].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [10]:
ws[0].todense().sum()

0

In [11]:
ws[332].todense().sum()

2